In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("/content/Crop and fertilizer dataset.csv")

# Encode labels
le_crop = LabelEncoder()
le_fert = LabelEncoder()
df['Crop'] = le_crop.fit_transform(df['Crop'])
df['Fertilizer'] = le_fert.fit_transform(df['Fertilizer'])

# ------------------ Crop Prediction ------------------
features = ['Nitrogen', 'Potassium', 'Phosphorus', 'Temperature', 'pH', 'Rainfall']
X_crop = df[features]
y_crop = df['Crop']

Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)
crop_model = RandomForestClassifier()
crop_model.fit(Xc_train, yc_train)

# ------------------ Fertilizer Prediction ------------------
df['CropEncoded'] = df['Crop']
X_fert = df[features + ['CropEncoded']]
y_fert = df['Fertilizer']

Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_fert, y_fert, test_size=0.2, random_state=42)
fert_model = RandomForestClassifier()
fert_model.fit(Xf_train, yf_train)

# ------------------ Ideal NPK Values (for rule-based advice) ------------------
ideal_npk = {
    'rice': (90, 40, 40),
    'wheat': (120, 60, 40),
    'maize': (140, 70, 40),
    'sugarcane': (100, 50, 50),
    'cotton': (80, 40, 40),
    'banana': (200, 150, 300),
    'groundnut': (25, 50, 75)
    # Add more if needed
}

def analyze_npk(crop, N, P, K):
    crop = crop.lower()
    if crop not in ideal_npk:
        return "No ideal NPK data available for this crop."

    ideal_N, ideal_P, ideal_K = ideal_npk[crop]
    advice = []

    if N < ideal_N:
        advice.append("Increase Nitrogen")
    elif N > ideal_N:
        advice.append("Decrease Nitrogen")

    if P < ideal_P:
        advice.append("Increase Phosphorous")
    elif P > ideal_P:
        advice.append("Decrease Phosphorous")

    if K < ideal_K:
        advice.append("Increase Potassium")
    elif K > ideal_K:
        advice.append("Decrease Potassium")

    return ", ".join(advice) if advice else "NPK levels are optimal."

# ------------------ Runtime Input ------------------
try:
    N = int(input("Enter Nitrogen content (N): "))
    P = int(input("Enter Phosphorous content (P): "))
    K = int(input("Enter Potassium content (K): "))
    temperature = float(input("Enter Temperature (°C): "))
    ph = float(input("Enter pH value of soil: "))
    rainfall = float(input("Enter Rainfall (mm): "))

    user_input = [[N, P, K, temperature, ph, rainfall]]

    # 1️⃣ Predict Crop
    crop_encoded = crop_model.predict(user_input)[0]
    crop_name = le_crop.inverse_transform([crop_encoded])[0]
    print(f"\nRecommended Crop: {crop_name}")

    # 2️⃣ Predict Fertilizer
    fert_input = [[N, P, K, temperature, ph, rainfall, crop_encoded]]
    fert_encoded = fert_model.predict(fert_input)[0]
    fert_name = le_fert.inverse_transform([fert_encoded])[0]
    print(f"Suggested Fertilizer: {fert_name}")

    # 3️⃣ Rule-based NPK Advice
    npk_advice = analyze_npk(crop_name, N, P, K)
    print(f"NPK Advice: {npk_advice}")

except Exception as e:
    print(" Error in input:", e)


Enter Nitrogen content (N): 80
Enter Phosphorous content (P): 45
Enter Potassium content (K): 86
Enter Temperature (°C): 5
Enter pH value of soil: 5
Enter Rainfall (mm): 45

Recommended Crop: Wheat
Suggested Fertilizer: 19:19:19 NPK
NPK Advice: Increase Nitrogen, Increase Phosphorous, Decrease Potassium


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
